In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np


# Read the CSV file
org_df = pd.read_csv('/content/drive/MyDrive/CUNY/Capstone/Data/market_price/e_mini_sp500_futuresHistorical_data_with_obtained_indexes_with_WSJ_20110801_20231006.csv')

# Convert 'Date' to datetime and set as index
org_df['Date'] = pd.to_datetime(org_df['Date'])
org_df.set_index('Date', inplace=True)

# Calculate absolute 'Change %'
org_df['up_down'] = np.where(org_df['Change %'] >= 0, 1, 0)



In [4]:
columns_stay = [
    # Y value
    'Close', 'up_down', 'previous_date_close_1',	'previous_date_close_2',	'previous_date_close_3',	'previous_date_close_4',	'previous_date_close_5',
    # Sentiment columns
    'negative', 'neutral', 'positive',
    'negative_weight', 'neutral_weight', 'positive_weight',
    'SA_simple_score', 'SA_weighted_score',
    'Positive_count', 'Neutral_count', 'Negative_count',

    # Simple sentiment percentages and ratios
    'positive_score_percentage_simple', 'negative_score_percentage_simple',
    'positive_negative_score_ratio',

    # Weighted sentiment percentages and ratios
    'positive_score_percentage_weighted', 'negative_score_percentage_weighted',
    'positive_negative_score_ratio_weighted',

    # Count-based percentages and ratios
    'positive_count_percentage', 'negative_count_percentage',
    'positive_negative_count_percentage_ratio',

    # Weighted count-based percentages and ratios
    'Positive_count_weighted', 'Neutral_count_weighted', 'Negative_count_weighted',
    'positive_count_percentage_weighted', 'negative_count_percentage_weighted',
    'positive_negative_count_percentage_ratio_weighted'
]



In [5]:
# Separate columns to stay and those to shift
cols_to_stay = org_df[columns_stay]
cols_to_shift = org_df.drop(columns_stay, axis=1)

# Shift the rows
shifted_cols = cols_to_shift.shift(periods=1, axis=0)

# Concatenate the columns side by side
corrected_df = pd.concat([cols_to_stay, shifted_cols], axis=1)
# Filter the DataFrame based on index values
corrected_df = corrected_df.loc['2012-01-03':'2023-01-03']

# Drop rows where both columns have a value of 0
mask = (corrected_df['positive_negative_score_ratio'] == 0) & (corrected_df['positive_negative_count_percentage_ratio'] == 0)
corrected_df = corrected_df[~mask]

print(corrected_df)

              Close  up_down  previous_date_close_1  previous_date_close_2  \
Date                                                                         
2012-01-03  1272.00        1                1252.50                1257.50   
2012-01-04  1273.00        1                1272.00                1252.50   
2012-01-05  1273.00        1                1273.00                1272.00   
2012-01-06  1274.25        1                1273.00                1273.00   
2012-01-09  1275.50        1                1274.25                1273.00   
...             ...      ...                    ...                    ...   
2022-12-27  3855.00        0                3893.50                3868.25   
2022-12-28  3807.50        0                3855.00                3893.50   
2022-12-29  3871.75        1                3807.50                3855.00   
2022-12-30  3861.00        0                3871.75                3807.50   
2023-01-03  3846.00        0                3871.50             

In [6]:
import pandas as pd
corrected_df.to_csv("/content/drive/MyDrive/CUNY/Capstone/Data/model/Data_set_for_training_and_testing_final.csv", index = True)